In [1]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/orco/Dokumente/PTB/MRS/MRSproc')


In [2]:
from mrsproc.preprocess import *
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
import suspect
import multiprocessing as mp
plt.rcParams['figure.figsize'] = [10, 7]

In [3]:
mm_files = glob.glob('/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-ad-special*.dat')

In [4]:
len(mm_files)

8

In [5]:
mm_files

['/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-702/mrs/sub-702_nuc-1H_loc-acc_spec-ad-special_TI1-800_TI2-42.dat',
 '/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-702/mrs/sub-702_nuc-1H_loc-pcc_spec-ad-special_TI1-800_TI2-42.dat',
 '/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-702/mrs/sub-702_nuc-1H_loc-wm_spec-ad-special_TI1-800_TI2-42.dat',
 '/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-703/mrs/sub-703_nuc-1H_loc-acc_spec-ad-special_TI1-800_TI2-42.dat',
 '/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-703/mrs/sub-703_nuc-1H_loc-pcc_spec-ad-special_TI1-800_TI2-42.dat',
 '/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-703/mrs/sub-703_nuc-1H_loc-wm_spec-ad-special_TI1-800_TI2-42.dat',
 '/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-701/mrs/sub-701_nuc-1H_loc-pcc_spec-ad-special_TI1-800_TI2-42_1.dat',
 '/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-701/mrs/sub-701_nuc-1H_loc-pcc_spec-ad-special_TI1-800_TI2-42_2.dat']

In [6]:
from joblib import Parallel, delayed
n_cpu = 4

In [9]:
%%time
with Parallel(n_jobs=2) as p:
    raw = p(delayed(suspect.io.load_twix)(i) for i in mm_files)

CPU times: user 952 ms, sys: 2.42 s, total: 3.38 s
Wall time: 29.2 s


In [10]:
raw[0].shape

(64, 32, 4096)

In [11]:
# The last twos are the sub.701 32 agerages each, they have to be merged
a1 = raw.pop(-1)
a2 = raw.pop(-1)

In [12]:
a = np.append(a1, a2, axis=0)

In [13]:
a.shape

(64, 32, 4096)

In [14]:
amrs = raw[0].inherit(a)

In [15]:
(amrs == a).all()

True

In [16]:
raw = raw + [amrs]

In [17]:
# Average every odd and even acquisition

In [18]:
def mean_everytwo(mm1):
    out = 0.5*(mm1[0::2] + mm1[1::2])
    return mm1.inherit(out)

In [19]:
%%time
with Parallel(n_jobs=2) as p:
    mm2 = p(delayed(mean_everytwo)(i) for i in raw)

CPU times: user 1.36 s, sys: 3.4 s, total: 4.76 s
Wall time: 6.08 s


In [20]:
[i.shape for i in mm2]

[(32, 32, 4096),
 (32, 32, 4096),
 (32, 32, 4096),
 (32, 32, 4096),
 (32, 32, 4096),
 (32, 32, 4096),
 (32, 32, 4096)]

In [21]:
def coil_combine(mrs):
    coil_combined = []
    for i in range(mrs.shape[0]):
        weights = suspect.processing.channel_combination.svd_weighting(mrs[i,:,:])
        coil_combined.append(suspect.processing.channel_combination.combine_channels(mrs[i,:,:],weights))
    return mrs.inherit(np.array(coil_combined))

In [22]:
%%time
coil_combined = [coil_combine(i) for i in mm2]

CPU times: user 38 s, sys: 35.2 s, total: 1min 13s
Wall time: 24.1 s


In [23]:
[i.shape for i in coil_combined]

[(32, 4096),
 (32, 4096),
 (32, 4096),
 (32, 4096),
 (32, 4096),
 (32, 4096),
 (32, 4096)]

In [24]:
[type(i) for i in coil_combined]

[suspect.mrsobjects.MRSData,
 suspect.mrsobjects.MRSData,
 suspect.mrsobjects.MRSData,
 suspect.mrsobjects.MRSData,
 suspect.mrsobjects.MRSData,
 suspect.mrsobjects.MRSData,
 suspect.mrsobjects.MRSData]

In [27]:
first = coil_combined[0]
first.shape

(32, 4096)

In [28]:
first.frequency_axis_ppm()

TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'

In [ ]:
first = coil_combined[0]
plt.plot(first.frequency_axis_ppm(), first.spectrum().real)
plt.xlim([5,0])
plt.axvline(4.7)

In [ ]:
corr = suspect.processing.frequency_correction.residual_water_alignment(first)
first = first.adjust_frequency(-corr)
corr